In [1]:
import os
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
from IPython.display import display, Markdown, Latex
from datetime import datetime

In [2]:
API_KEY = os.environ['MISTRAL_API_KEY']

MODEL_ID = 'open-mixtral-8x22b'
MODEL_SEED = 2
MODEL_TEMPERATURE = 0.7

file_system_prompt_1 = open("../../system_prompt_1.txt", "r")
file_system_prompt_2 = open("../../system_prompt_2.txt", "r")
SYSTEM_PROMPT_1 = file_system_prompt_1.read()
SYSTEM_PROMPT_2 = file_system_prompt_2.read()
file_system_prompt_1.close()
file_system_prompt_2.close()

FILE_PATH_PROBLEM = '../../../../Datasets/MIP_1_Facility_Location_Allocation/'
assert(FILE_PATH_PROBLEM != '../../../../Datasets/')
file_problem_description = open(FILE_PATH_PROBLEM + 'ProblemDescription.txt', 'r')
PROBLEM_DESCRIPTION = file_problem_description.read()
file_problem_description.close()


client = MistralClient(api_key=API_KEY)

print(f'Time of execution: {datetime.now()}')

Time of execution: 2024-05-18 12:11:21.008696


## Step 1 - Generate Mathematical Formulation 

In [3]:
messages_1 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_1),
    ChatMessage(role="user", content=PROBLEM_DESCRIPTION)
]

In [4]:
response_1 = client.chat(
    model=MODEL_ID,
    messages=messages_1,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE 
)

response_1_text = response_1.choices[0].message.content

In [11]:
Markdown(response_1_text)

To formulate a mathematical optimization model for this problem, we will use the following parameters, decision variables, objective function, and constraints:

Parameters:

- Let $I$ be the set of possible locations for opening facilities.
- Let $J$ be the set of customers.
- Let $f_i$ be the fixed annual activation cost for opening a facility at location $i \in I$.
- Let $s_i$ be the maximum annual service volume at facility $i \in I$.
- Let $d_j$ be the annual demand of customer $j \in J$.
- Let $c_{ij}$ be the transportation cost for servicing customer $j \in J$ from facility $i \in I$.

Decision variables:

- Let $y_i$ be a binary variable equal to 1 if a facility is opened at location $i \in I$, and 0 otherwise.
- Let $x_{ij}$ be a binary variable equal to 1 if customer $j \in J$ is serviced by facility $i \in I$, and 0 otherwise.

Objective function:

The objective is to minimize the overall costs, represented by the sum of the fixed activation costs for any opened facilities and the transportation costs for servicing customers. Therefore, the objective function is:

\[
\min \sum_{i \in I} f_i y_i + \sum_{i \in I} \sum_{j \in J} c_{ij} x_{ij}
\]

Constraints:

1. Each customer's demand must be met:
\[
\sum_{i \in I} x_{ij} = 1 \quad \forall j \in J
\]

2. Each facility cannot exceed its maximum service volume:
\[
\sum_{j \in J} d_j x_{ij} \leq s_i y_i \quad \forall i \in I
\]

3. Customers can only be serviced by an opened facility:
\[
x_{ij} \leq y_i \quad \forall i \in I, j \in J
\]

4. Non-negativity and binary constraints:
\[
x_{ij} \in \{0, 1\} \quad \forall i \in I, j \in J
\]
\[
y_i \in \{0, 1\} \quad \forall i \in I
\]

This formulation is a mixed-integer linear programming (MILP) problem, which can be solved using standard optimization techniques.

In [6]:
print(response_1_text)

To formulate a mathematical optimization model for this problem, we will use the following parameters, decision variables, objective function, and constraints:

Parameters:

- Let $I$ be the set of possible locations for opening facilities.
- Let $J$ be the set of customers.
- Let $f_i$ be the fixed annual activation cost for opening a facility at location $i \in I$.
- Let $s_i$ be the maximum annual service volume at facility $i \in I$.
- Let $d_j$ be the annual demand of customer $j \in J$.
- Let $c_{ij}$ be the transportation cost for servicing customer $j \in J$ from facility $i \in I$.

Decision variables:

- Let $y_i$ be a binary variable equal to 1 if a facility is opened at location $i \in I$, and 0 otherwise.
- Let $x_{ij}$ be a binary variable equal to 1 if customer $j \in J$ is serviced by facility $i \in I$, and 0 otherwise.

Objective function:

The objective is to minimize the overall costs, represented by the sum of the fixed activation costs for any opened facilities an

## Step 2 - Generate the Pyomo Code

In [7]:
messages_2 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_2),
    ChatMessage(role="user", content=response_1_text)
]

In [8]:
response_2 = client.chat(
    model=MODEL_ID,
    messages=messages_2,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE
)

response_2_text = response_2.choices[0].message.content

In [9]:
Markdown(response_2_text)

To solve this optimization problem using Pyomo, you will first need to install the Pyomo package. You can do this by running `pip install pyomo` in your terminal. Once you have Pyomo installed, you can use the following code as a starting point:

```python
import pyomo.environ as pyo

# Sample data
I = range(1, 4)  # Set of possible locations for opening facilities
J = range(1, 6)  # Set of customers
f = {1: 1000, 2: 1200, 3: 1100}  # Fixed annual activation cost for opening a facility
s = {1: 10000, 2: 12000, 3: 11000}  # Maximum annual service volume at a facility
d = {1: 1000, 2: 2000, 3: 1500, 4: 1800, 5: 2200}  # Annual demand of customers
c = {  # Transportation cost for servicing a customer from a facility
    (1, 1): 50, (1, 2): 60, (1, 3): 55, (1, 4): 70, (1, 5): 80,
    (2, 1): 70, (2, 2): 65, (2, 3): 75, (2, 4): 60, (2, 5): 90,
    (3, 1): 60, (3, 2): 50, (3, 3): 65, (3, 4): 80, (3, 5): 70
}

model = pyo.ConcreteModel()

# Define decision variables
model.y = pyo.Var(I, within=pyo.Binary)  # Binary variable for opening a facility
model.x = pyo.Var(I, J, within=pyo.Binary)  # Binary variable for servicing a customer

# Objective function
def obj_rule(model):
    return sum(f[i] * model.y[i] for i in I) + sum(c[i, j] * model.x[i, j] for i in I for j in J)

model.obj = pyo.Objective(rule=obj_rule, sense=pyo.minimize)

# Constraints
def demand_met_rule(model, j):
    return sum(model.x[i, j] for i in I) == 1

model.demand_met = pyo.Constraint(J, rule=demand_met_rule)

def max_service_volume_rule(model, i):
    return sum(d[j] * model.x[i, j] for j in J) <= s[i] * model.y[i]

model.max_service_volume = pyo.Constraint(I, rule=max_service_volume_rule)

def serviced_by_opened_facility_rule(model, i, j):
    return model.x[i, j] <= model.y[i]

model.serviced_by_opened_facility = pyo.Constraint(I, J, rule=serviced_by_opened_facility_rule)

# Solve the model
solver = pyo.SolverFactory('glpk')  # You can use any installed MILP solver
solver.solve(model)

# Print the results
for i in I:
    print(f"Open facility at location {i}: {model.y[i].value}")
    for j in J:
        print(f"  Service customer {j} from location {i}: {model.x[i, j].value}")
print(f"Objective value: {model.obj()}")
```

This code uses the GLPK solver to solve the optimization problem. You can replace it with any other MILP solver that you have installed. The sample data provided in the code is just an example, and you can replace it with your actual data.

In [10]:
print(response_2_text)

To solve this optimization problem using Pyomo, you will first need to install the Pyomo package. You can do this by running `pip install pyomo` in your terminal. Once you have Pyomo installed, you can use the following code as a starting point:

```python
import pyomo.environ as pyo

# Sample data
I = range(1, 4)  # Set of possible locations for opening facilities
J = range(1, 6)  # Set of customers
f = {1: 1000, 2: 1200, 3: 1100}  # Fixed annual activation cost for opening a facility
s = {1: 10000, 2: 12000, 3: 11000}  # Maximum annual service volume at a facility
d = {1: 1000, 2: 2000, 3: 1500, 4: 1800, 5: 2200}  # Annual demand of customers
c = {  # Transportation cost for servicing a customer from a facility
    (1, 1): 50, (1, 2): 60, (1, 3): 55, (1, 4): 70, (1, 5): 80,
    (2, 1): 70, (2, 2): 65, (2, 3): 75, (2, 4): 60, (2, 5): 90,
    (3, 1): 60, (3, 2): 50, (3, 3): 65, (3, 4): 80, (3, 5): 70
}

model = pyo.ConcreteModel()

# Define decision variables
model.y = pyo.Var(I, withi

### Code Executability

In [12]:
import pyomo.environ as pyo

# Sample data
I = range(1, 4)  # Set of possible locations for opening facilities
J = range(1, 6)  # Set of customers
f = {1: 1000, 2: 1200, 3: 1100}  # Fixed annual activation cost for opening a facility
s = {1: 10000, 2: 12000, 3: 11000}  # Maximum annual service volume at a facility
d = {1: 1000, 2: 2000, 3: 1500, 4: 1800, 5: 2200}  # Annual demand of customers
c = {  # Transportation cost for servicing a customer from a facility
    (1, 1): 50, (1, 2): 60, (1, 3): 55, (1, 4): 70, (1, 5): 80,
    (2, 1): 70, (2, 2): 65, (2, 3): 75, (2, 4): 60, (2, 5): 90,
    (3, 1): 60, (3, 2): 50, (3, 3): 65, (3, 4): 80, (3, 5): 70
}

model = pyo.ConcreteModel()

# Define decision variables
model.y = pyo.Var(I, within=pyo.Binary)  # Binary variable for opening a facility
model.x = pyo.Var(I, J, within=pyo.Binary)  # Binary variable for servicing a customer

# Objective function
def obj_rule(model):
    return sum(f[i] * model.y[i] for i in I) + sum(c[i, j] * model.x[i, j] for i in I for j in J)

model.obj = pyo.Objective(rule=obj_rule, sense=pyo.minimize)

# Constraints
def demand_met_rule(model, j):
    return sum(model.x[i, j] for i in I) == 1

model.demand_met = pyo.Constraint(J, rule=demand_met_rule)

def max_service_volume_rule(model, i):
    return sum(d[j] * model.x[i, j] for j in J) <= s[i] * model.y[i]

model.max_service_volume = pyo.Constraint(I, rule=max_service_volume_rule)

def serviced_by_opened_facility_rule(model, i, j):
    return model.x[i, j] <= model.y[i]

model.serviced_by_opened_facility = pyo.Constraint(I, J, rule=serviced_by_opened_facility_rule)

# Solve the model
solver = pyo.SolverFactory('glpk')  # You can use any installed MILP solver
solver.solve(model)

# Print the results
for i in I:
    print(f"Open facility at location {i}: {model.y[i].value}")
    for j in J:
        print(f"  Service customer {j} from location {i}: {model.x[i, j].value}")
print(f"Objective value: {model.obj()}")

Open facility at location 1: 1.0
  Service customer 1 from location 1: 1.0
  Service customer 2 from location 1: 1.0
  Service customer 3 from location 1: 1.0
  Service customer 4 from location 1: 1.0
  Service customer 5 from location 1: 1.0
Open facility at location 2: 0.0
  Service customer 1 from location 2: 0.0
  Service customer 2 from location 2: 0.0
  Service customer 3 from location 2: 0.0
  Service customer 4 from location 2: 0.0
  Service customer 5 from location 2: 0.0
Open facility at location 3: 0.0
  Service customer 1 from location 3: 0.0
  Service customer 2 from location 3: 0.0
  Service customer 3 from location 3: 0.0
  Service customer 4 from location 3: 0.0
  Service customer 5 from location 3: 0.0
Objective value: 1315.0


### Solution Correctness

In [14]:
import pyomo.environ as pyo

# Sample data
I = range(1, 4)  # Set of possible locations for opening facilities
J = range(1, 6)  # Set of customers
f = {1: 1000, 2: 1000, 3: 1000}  # Fixed annual activation cost for opening a facility
s = {1: 500, 2: 500, 3: 500}  # Maximum annual service volume at a facility
d = {1: 80, 2: 270, 3: 250, 4: 160, 5: 180}  # Annual demand of customers
c = {  # Transportation cost for servicing a customer from a facility
    (1, 1): 4,  (1, 2): 6,  (1, 3): 9,
    (2, 1): 5,  (2, 2): 4,  (2, 3): 7,
    (3, 1): 6,  (3, 2): 3,  (3, 3): 4,
    (4, 1): 8,  (4, 2): 5,  (4, 3): 3,
    (5, 1): 10, (5, 2): 8,  (5, 3): 4,
}

model = pyo.ConcreteModel()

# Define decision variables
model.y = pyo.Var(I, within=pyo.Binary)  # Binary variable for opening a facility
model.x = pyo.Var(I, J, within=pyo.Binary)  # Binary variable for servicing a customer

# Objective function
def obj_rule(model):
    return sum(f[i] * model.y[i] for i in I) + sum(c[j, i] * model.x[i, j] for i in I for j in J)

model.obj = pyo.Objective(rule=obj_rule, sense=pyo.minimize)

# Constraints
def demand_met_rule(model, j):
    return sum(model.x[i, j] for i in I) == 1

model.demand_met = pyo.Constraint(J, rule=demand_met_rule)

def max_service_volume_rule(model, i):
    return sum(d[j] * model.x[i, j] for j in J) <= s[i] * model.y[i]

model.max_service_volume = pyo.Constraint(I, rule=max_service_volume_rule)

def serviced_by_opened_facility_rule(model, i, j):
    return model.x[i, j] <= model.y[i]

model.serviced_by_opened_facility = pyo.Constraint(I, J, rule=serviced_by_opened_facility_rule)

# Solve the model
solver = pyo.SolverFactory('glpk')  # You can use any installed MILP solver
solver.solve(model)

# Print the results
for i in I:
    print(f"Open facility at location {i}: {model.y[i].value}")
    for j in J:
        print(f"  Service customer {j} from location {i}: {model.x[i, j].value}")
print(f"Objective value: {model.obj()}")

Open facility at location 1: 0.0
  Service customer 1 from location 1: 0.0
  Service customer 2 from location 1: 0.0
  Service customer 3 from location 1: 0.0
  Service customer 4 from location 1: 0.0
  Service customer 5 from location 1: 0.0
Open facility at location 2: 1.0
  Service customer 1 from location 2: 1.0
  Service customer 2 from location 2: 0.0
  Service customer 3 from location 2: 1.0
  Service customer 4 from location 2: 1.0
  Service customer 5 from location 2: 0.0
Open facility at location 3: 1.0
  Service customer 1 from location 3: 0.0
  Service customer 2 from location 3: 1.0
  Service customer 3 from location 3: 0.0
  Service customer 4 from location 3: 0.0
  Service customer 5 from location 3: 1.0
Objective value: 2025.0
